In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

def blight_model(): 
    
    
    X_train = pd.read_csv("train.csv", encoding = 'ISO-8859-1' )
    X_test = pd.read_csv("test.csv", encoding = 'ISO-8859-1' )
    
    # Preprocessing of data sets

    X_train = X_train[(X_train['compliance'] == 0) | (X_train['compliance'] == 1)]
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    # Joining addresses to latitude and longitude

    addresses = addresses.set_index('address')
    latlons = latlons.set_index('address')
    addresses = addresses.join(latlons, how='left')
    addresses = addresses.set_index('ticket_id')
    
    # Joining to train and test data

    X_train = X_train.set_index('ticket_id')
    X_test = X_test.set_index('ticket_id')
    X_train = X_train.join(addresses, how='left')
    X_test = X_test.join(addresses, how='left') 
    y_train = X_train[['compliance']]
    
    # Filling NULL values

    X_train.lat.fillna(method='pad', inplace=True)
    X_train.lon.fillna(method='pad', inplace=True)
    X_train.state.fillna(method='pad', inplace=True)

    X_test.lat.fillna(method='pad', inplace=True)
    X_test.lon.fillna(method='pad', inplace=True)
    
    # Data Leakage 

    train_leaks = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status', 
        'compliance'
    ]
    
    # Columns with no information

    no_info_columns = ['agency_name', 'violation_street_number', 'mailing_address_str_number',
                      'state', 'disposition', 'violator_name', 'zip_code', 'country', 'city',
                      'inspector_name', 'violation_street_name', 'violation_zip_code',
                      'violation_description', 'mailing_address_str_name', 'non_us_str_code',
                      'ticket_issued_date', 'hearing_date', 'violation_code', 'grafitti_status']
    
    X_train.drop(train_leaks, inplace=True, axis=1)
    X_train.drop(no_info_columns, inplace= True, axis=1)
    X_test.drop(no_info_columns, inplace=True, axis=1)
    
    clfLR = LogisticRegression().fit(X_train, y_train)
    # regr_rf = RandomForestRegressor()
    
    test_proba = clfLR.predict_proba(X_test)[:,1]
    X_test['compliance'] = test_proba
        
    # Using Grid Search to find optimal values for finding optimal values

    #grid_values = {'C': [0.001]}
    #grid_clf_auc = GridSearchCV(clfLR, param_grid = grid_values, scoring = 'roc_auc')
    #grid_clf_auc.fit(X_train, y_train)
    
    
    # grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
    # grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
    # grid_clf_auc.fit(X_train, y_train)
    # print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    # print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    
    return X_test.compliance


blight_model()

ticket_id
284932    0.057993
285362    0.007384
285361    0.075421
285338    0.058295
285346    0.074732
            ...   
376496    0.007332
376497    0.007332
376499    0.074383
376500    0.074382
369851    0.112515
Name: compliance, Length: 61001, dtype: float64

In [3]:
blight_model()

ticket_id
284932    0.057993
285362    0.007384
285361    0.075421
285338    0.058295
285346    0.074732
            ...   
376496    0.007332
376497    0.007332
376499    0.074383
376500    0.074382
369851    0.112515
Name: compliance, Length: 61001, dtype: float64

In [5]:
from sklearn.model_selection import cross_val_score
# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(clfLR, X_train, y_train, cv=5))



NameError: name 'clfLR' is not defined